In [1]:
%pip install -qU langchain langchain_community langchain_huggingface langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -qU python-dotenv

from dotenv import load_dotenv
import os

# loads the .env file (if you have a global environment variable, you can skip this)
load_dotenv()

# lets just validate that we have the environment variable
api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import InMemoryVectorStore

loader = WebBaseLoader("https://www.govinfo.gov/content/pkg/CDOC-110hdoc50/html/CDOC-110hdoc50.htm")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n\n", "\n", " ", ""])
chunks = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = InMemoryVectorStore.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Users\sekkei35\Desktop\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# setup the chat model and prompt template
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub
llm = ChatOpenAI(model_name="gpt-4o")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can answer questions about the US Constitution. Use the provided context to answer the question. IMPORTANT: If you are unsure of the answer, say 'I don't know' and don't make up an answer."),
    ("user", "Question: {question}\nContext: {context}")
])
prompt = hub.pull("rlm/rag-prompt")
chain = prompt | llm

c:\Users\sekkei35\Desktop\RAG\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
query = "Is there any plans to change the US Constitution to allow for term limits on senators?"

# RETRIEVAL
docs = retriever.invoke(query)
docs_content = "\n\n".join(doc.page_content for doc in docs)

# AUGMENTED + GENERATION
response = chain.invoke({"question": query, "context": docs})

print(response.content)

In [ ]:
while True:
    query = input("Enter a question: ")
    if query == "exit":
        break
    docs = retriever.invoke(query)
    response = chain.invoke({"question": query, "context": docs})
    print(response.content)